
$\textbf{Problema Primal}$ 
\
\
$Si \  se \  tiene \  n \  estaciones, \  N=\{1,...,n\}, \  K \  todas \  las \  rutas \  posibles \  $
\
\
$\ \displaystyle \min \ \ \sum_{o \in N}\sum_{d \in N}\sum_{i \in N}\sum_{j \in N}c_{ij}x_{ij}^{od}+\alpha \sum_{k \in K}f_{k}$
\
\
$Restricciones$
\
\
$\displaystyle\sum_{j \in N} x_{ij}^{od} -\sum_{j \in N} x_{ji}^{od}=b_{i}^{od} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall  i \in N, \ \forall  o \in N, \ \forall  d \in N$ 

$\displaystyle\sum_{o \in N}\sum_{d \in N} x_{ij}^{od} \leq \sum_{k \in K}u*r_{kij}*f_{k} \ \ \ \ \ \ \ \ \forall  i \in N, \ \forall  j \in N $
\
\
$\displaystyle\sum_{k \in K} f_{k} \leq Q$
\
\
$x_{ji}^{od}\geq 0, \ f_{k}\geq 0 $
\
\
$x_{ji}^{od} \ representa \ el \ flujo \ entre \ las \ estaciones \ i \ y \ j \ para \ satisfacer \ la \ demanda  \ o  \ d$
$f_{k} \ representa  \ la \  cantidad \  de \  buses \  de \ la \ ruta \  k  \ que  \ sera  \ usada  \ para  \ satisfacer  \ las \  demandas$ 



$\textbf{Problema Dual}$ 
\
\
$max \ \ \lambda Q + \sum_{i \in N} \sum_{o \in N} \sum_{d \in N}b_{i}^{od}\alpha_{i}^{od} $
\
\
$Restricciones$
\
\
$\displaystyle\alpha _{i}^{od} -\alpha _{j}^{od} + \beta _{ij} \leq c_{ij}\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall  i \in N, \ \forall  j \in N, \forall  o \in N, \ \forall  d \in N  $

$\displaystyle\lambda - \sum_{i \in N } \sum_{ j \in N} r_{kij}* \beta_{ij}*u \leq \alpha \ \ \ \ \ \ \ \ \forall  k \in K$
\
\
$\beta_{ij} \leq 0 ,\lambda \leq 0 $
\
\
$\alpha_{i}^{od} \ son \  las  \ variable \  duales \ de  \ las  \ restricciones  \ de \  conservación \  de  \ flujo \ $
\
$\beta_{ij} \  son \  las \  variable  \ duales  \ de \  las \  restricciones \  de \  capacidad $
\
$\lambda \ es  \ la \  variable \  dual  \ de  \ la  \ restriccion \  de \  maximo  \ numero  \ de \  buses $
\
\
$\textbf{Generación de rutas}$ 
\
\
$ max \ \ \lambda-\alpha- \sum_{i \in N,j \in N}r_{ij}\beta_{ij}u$
\
\
$Restricciones$
\
\
$ \displaystyle\sum_{j=2}^{n}r_{1j} \leq s_{1}$
\
$\displaystyle\sum_{j=i+1}^{n}r_{ij} \leq s_{i}+ \sum_{k=1}^{i-1}r_{ki}, \ \forall i \in N-\{1\}  $
\
$\displaystyle\sum_{i=1}^{n} s_{i} \leq 1 $
\
$r_{ij}=r_{ji} \ \forall i \in N, \forall j \in N$
\
$r_{ii}=0, \ \forall i \in N$
\
\
\
$r_{ij} \in \{0,1 \}, \ s_{i} \in \{0,1 \}$

In [2]:


numNodos=15
rutas=[]
rutaAux=[]
alpha=10
u= 100
m= 40
Q= 50
respuesta=alpha+1.0
for i in range(numNodos):
    rutaAux.append(1)
rutas.append(rutaAux)    
iterador=0
while(respuesta>0.00001):
    
    numRutas=len(rutas)
    
    #contruir Bod
    b=[]
    aux=[]
    for i in range(numNodos):
         aux.append(0)         
    for i in range(numNodos):
         b.append(aux[:])  
    for i in range(numNodos):
        for j in range(numNodos):
            if(i != j):
                b[i][j]=50
    #contruir Cij                       
    c=[]
    aux=[]
    for i in range(numNodos):
         aux.append(0) 
    for i in range(numNodos):
         c.append(aux[:])  
    for i in range(numNodos):
        for j in range(numNodos):
            if(i != j):
                c[i][j]=abs(i-j)*10-abs(i-j)+1 
    # construir Bodi  

    B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
    for i in range(numNodos):
        for j in range(numNodos):
            B2[i,j,i]=b[i][j]
            B2[i,j,j]=-b[i][j]


    arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]

    #---------------------
    

    numRutas=len(rutas)
    rutasmodificadas=[]
    rutaaux=[]
    for i in range(numRutas):
       rutaaux.clear() 
       for j in range(numNodos):
            if rutas[i][j] == 1:
                rutaaux.append(j)          
       rutasmodificadas.append(rutaaux[:])  
    r={}
    for i in arcos1:
        r[i]=0
    for i in range(numRutas):
        for j in range(len(rutasmodificadas[i])-1):
            r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
            r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
    #-------------------------

    arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
    arcosY=[(i) for i in range (numRutas)]

    

    from docplex.mp.model import Model


    mdl= Model("BRT")
    #creacion variables de desicion
    x=mdl.continuous_var_dict(arcosX,name="x")
    y=mdl.continuous_var_dict(arcosY,name="y")


     # funcion a minimizar
    costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
    costoY=mdl.sum((alpha*y[i]) for i in range(numRutas))
    obj=mdl.minimize(costoX+costoY)



    #capacidad  
    capacidad=[]
    for i in  range(numNodos):
        for j in range(numNodos): 
            capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))

    #flujo
    flujo=[]
    for i in range(numNodos):  
        for o in range(numNodos):
            for d in range(numNodos):       
                flujo.append(mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)]))
    #maximo rutas

    maxRutas=mdl.add_constraint( mdl.sum(y[(k)]for k in range(numRutas))<=m)  


    mdl.solve(log_output=False)
    matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
    vector=[(i) for i in range(numNodos)]
    r={}
    for i in matriz:
        r[i]=0




    mdlAux= Model("Aux")
    rutaNueva=mdlAux.binary_var_dict(matriz,name="r")
    aux=mdlAux.binary_var_dict(vector,name="s")
    dualCapacidad = mdl.dual_values(capacidad)

    #print(mdl.dual_values(flujo))
    dualMaxRutas=mdl._dual_value1(maxRutas)
    objAux=mdlAux.maximize(dualMaxRutas-alpha+mdlAux.sum((-rutaNueva[i,j]*u)*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))
   
    

    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=aux[(0)])

    for i in range(1,numNodos):    
        mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1))+aux[(i)])

    for i in range(numNodos):
        for j in range(i+1,numNodos):
            mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])

    for i in range(numNodos):
        mdlAux.add_constraint(rutaNueva[(i,i)]==0)

    mdlAux.add_constraint(mdlAux.sum(aux[(i)] for i in range(numNodos) )<=1)

    mdlAux.solve(log_output=False)
    nuevaRuta=[]
    for i in range(numNodos):
        nuevaRuta.append(0)
    for i in range(numNodos):
        for j in range (numNodos):
            if(rutaNueva[(i,j)].solution_value==1):
                nuevaRuta[i]=1
                nuevaRuta[j]=1
    rutas.append(nuevaRuta)
    respuesta=mdlAux.objective_value
    print("ruta nueva",nuevaRuta,"valor objetivo ",mdl.objective_value,"valor dual",respuesta )
    iterador=iterador+1

solucion=mdl.solve(log_output=True)  
archivo = open("solucion2", "w") 
archivo.write(str(solucion)) 
archivo.close() 
   

ruta nueva [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] valor objetivo  560280.0 valor dual 2599.9999999999995
ruta nueva [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] valor objetivo  558980.0 valor dual 2400.0
ruta nueva [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] valor objetivo  552410.0 valor dual 2200.0
ruta nueva [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] valor objetivo  548025.0 valor dual 2200.0
ruta nueva [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1] valor objetivo  546635.0 valor dual 2010.0000000000002
ruta nueva [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] valor objetivo  543860.0 valor dual 2000.0
ruta nueva [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] valor objetivo  542860.0 valor dual 2000.0
ruta nueva [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1] valor objetivo  541860.0 valor dual 1810.0000000000005
ruta nueva [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1] valor objetivo  538800.0 valor dual 2000.0000000000007
ruta nueva [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1

KeyboardInterrupt: 

In [78]:
for i in range(numRutas):
        print(i,rutas[i],y[(i)].solution_value)

0 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 0.5
1 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.5
2 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] 0.5
3 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1] 0.5
4 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.5
5 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.5
6 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1] 0.5
7 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1] 0.5
8 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1] 0.5
9 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1] 0
10 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
11 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] 0.5
12 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
13 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1] 0.5
14 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0] 0.5
15 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
16 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0] 0.5
17 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0] 0.5
18 [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0] 0.5
19 [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0] 0.5
20 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0] 0
21 [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0] 0.5
22 [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0] 0.5
23

In [60]:
for i in range(numNodos):
    for j in range(numNodos):
        print(capacidad[numNodos*i+j])

x_0_0_0_0+x_0_0_0_1+x_0_0_0_2+x_0_0_0_3+x_0_0_1_0+x_0_0_1_1+x_0_0_1_2+x_0_0_1_3+x_0_0_2_0+x_0_0_2_1+x_0_0_2_2+x_0_0_2_3+x_0_0_3_0+x_0_0_3_1+x_0_0_3_2+x_0_0_3_3 <= 0
x_0_1_0_0+x_0_1_0_1+x_0_1_0_2+x_0_1_0_3+x_0_1_1_0+x_0_1_1_1+x_0_1_1_2+x_0_1_1_3+x_0_1_2_0+x_0_1_2_1+x_0_1_2_2+x_0_1_2_3+x_0_1_3_0+x_0_1_3_1+x_0_1_3_2+x_0_1_3_3 <= 100y_0
x_0_2_0_0+x_0_2_0_1+x_0_2_0_2+x_0_2_0_3+x_0_2_1_0+x_0_2_1_1+x_0_2_1_2+x_0_2_1_3+x_0_2_2_0+x_0_2_2_1+x_0_2_2_2+x_0_2_2_3+x_0_2_3_0+x_0_2_3_1+x_0_2_3_2+x_0_2_3_3 <= 100y_2
x_0_3_0_0+x_0_3_0_1+x_0_3_0_2+x_0_3_0_3+x_0_3_1_0+x_0_3_1_1+x_0_3_1_2+x_0_3_1_3+x_0_3_2_0+x_0_3_2_1+x_0_3_2_2+x_0_3_2_3+x_0_3_3_0+x_0_3_3_1+x_0_3_3_2+x_0_3_3_3 <= 100y_1
x_1_0_0_0+x_1_0_0_1+x_1_0_0_2+x_1_0_0_3+x_1_0_1_0+x_1_0_1_1+x_1_0_1_2+x_1_0_1_3+x_1_0_2_0+x_1_0_2_1+x_1_0_2_2+x_1_0_2_3+x_1_0_3_0+x_1_0_3_1+x_1_0_3_2+x_1_0_3_3 <= 100y_0
x_1_1_0_0+x_1_1_0_1+x_1_1_0_2+x_1_1_0_3+x_1_1_1_0+x_1_1_1_1+x_1_1_1_2+x_1_1_1_3+x_1_1_2_0+x_1_1_2_1+x_1_1_2_2+x_1_1_2_3+x_1_1_3_0+x_1_1_3_1+x_1_1_3_2+x_1_1

In [2]:
for i in range(numRutas):
    if(y[(i)].solution_value):
        print(rutas[i],y[(i)].solution_value)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 0.5
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.5
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1] 0.5
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] 0.5
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1] 0.5
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0] 0.5
[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 